In [44]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

In [45]:
DataPath = 'Data'

In [46]:
OuptutPath = 'Output'
os.makedirs(OuptutPath, exist_ok=True)

In [47]:
# Loading all the images from input directory
images = [cv2.imread(os.path.join(DataPath, f)) for f in os.listdir(DataPath) if f.endswith('.jpg')]

In [48]:
depth_to_print = 3
curvature_threshold = 0.02
Curve_comparison = 4
# find contour of the image up to 4 level hierarchy
for i, img in enumerate(images):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray, 127, 255, 0)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    img_contour = img

    for j in range(len(contours)):
        level = 0
        parent = hierarchy[0][j][3]
        
        # Traverse up the hierarchy to calculate the depth of the contour
        while parent != -1:
            level += 1
            parent = hierarchy[0][parent][3]
        
        # If contour is deeper than allowed depth, ignore it
        if level > depth_to_print:
            continue
        
        # Approximate the contour to see if it's curved
        epsilon = curvature_threshold * cv2.arcLength(contours[j], True)
        approx = cv2.approxPolyDP(contours[j], epsilon, True)
        
        # Ignore highly curved contours (if the approximation has too many points)
        if len(approx) > Curve_comparison:  
            continue
        
        # Draw the contour if it passes the depth and curvature checks
        cv2.drawContours(img_contour, [contours[j]], -1, (0, 0, 255), 3)

    cv2.imwrite(os.path.join(OuptutPath, f'output_{i}.jpg'), img_contour)

In [49]:
# Display the images
for i, img in enumerate(images):
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')

In [ ]:
# aplying Segmentation on the images to seperate the objects